# PRIO Linux: IIC Demo
This demo illustrates how to use device tree overlay's and partial reconfiguration to interact with the grove haptic motor via IIC.

### Step One:

Download the static bitstream

In [1]:
from prio_linux import PrIoOverlay

FULL_BITSTREAM_PATH = "/usr/local/lib/python3.6/dist-packages/prio_linux/"
PARTIAL_BITSTREAM_PATH = "/usr/local/lib/python3.6/dist-packages/prio_linux/partial/"
DTBO_PATH = "/usr/local/lib/python3.6/dist-packages/prio_linux/dtbo/"

overlay = PrIoOverlay(FULL_BITSTREAM_PATH + "prio_linux.bit")

### Step Two:
Download the partial bitstream and insert the dtbo files  
**Note:** Currently the IIC linux driver does not support being removed. This means that you must restart your board between each run of this 

In [2]:
overlay.pr_download("pr_2", PARTIAL_BITSTREAM_PATH + "pr_2_iic.bit", DTBO_PATH + "pr_2_iic.dtbo")

### Step Three: 
Define the Haptic class. This class is a convenient way to set up and communicate with the haptic motor. I2C linux driver 

Note that this class requires the use of the smbus2 libary, which is a python wrapper around the I2C linux driver associated with the dtbo file inserted in step 2. If you have not already installed this library then run "pip3 install smbus2" in the terminal.

In [3]:
from smbus2 import SMBus
        
class Haptic:
    STATUS_Reg          = 0x00
    MODE_Reg            = 0x01
    RTP_INPUT_Reg       = 0x02
    LIB_SEL_Reg         = 0x03
    WAV_SEQ1_Reg        = 0x04
    WAV_SEQ2_Reg        = 0x05
    WAV_SEQ3_Reg        = 0x06
    WAV_SEQ4_Reg        = 0x07
    WAV_SEQ5_Reg        = 0x08
    WAV_SEQ6_Reg        = 0x09
    WAV_SEQ7_Reg        = 0x0A
    WAV_SEQ8_Reg        = 0x0B
    GO_Reg              = 0x0C
    ODT_OFFSET_Reg      = 0x0D
    SPT_Reg             = 0x0E
    SNT_Reg             = 0x0F
    BRT_Reg             = 0x10
    ATV_CON_Reg         = 0x11
    ATV_MIN_IN_Reg      = 0x12
    ATV_MAX_IN_Reg      = 0x13
    ATV_MIN_OUT_Reg     = 0x14
    ATV_MAX_OUT_Reg     = 0x15
    RATED_VOLTAGE_Reg   = 0x16
    OD_CLAMP_Reg        = 0x17
    A_CAL_COMP_Reg      = 0x18
    A_CAL_BEMF_Reg      = 0x19
    FB_CON_Reg          = 0x1A
    CONTRL1_Reg         = 0x1B
    CONTRL2_Reg         = 0x1C
    CONTRL3_Reg         = 0x1D
    CONTRL4_Reg         = 0x1E
    VBAT_MON_Reg        = 0x21
    LRA_RESON_Reg       = 0x22
    
    def __init__(self, bus_num, haptic_address):
        
        # set the bus number and address of the haptic motor
        self.bus = SMBus(bus_num)
        self.address = haptic_address
        
        # Enter standby mode
        self.bus.write_byte_data(self.address, self.MODE_Reg, 0x40)
        time.sleep(0.25)

        self.bus.write_byte_data(self.address, self.RATED_VOLTAGE_Reg, 0x50)

        # Set overdrive voltage 
        self.bus.write_byte_data(self.address, self.OD_CLAMP_Reg, 0x89)

        # Setup feedback control and control registers
        self.bus.write_byte_data(self.address, self.FB_CON_Reg, 0xB6)
        self.bus.write_byte_data(self.address, self.CONTRL1_Reg, 0x13)
        self.bus.write_byte_data(self.address, self.CONTRL2_Reg, 0xF5)
        self.bus.write_byte_data(self.address, self.CONTRL3_Reg, 0x80)

        # Select the LRA Library 
        self.bus.write_byte_data(self.address, self.LIB_SEL_Reg, 0x06)

        # Put the DRV2605 device in active mode
        self.bus.write_byte_data(self.address, self.MODE_Reg, 0x00)

    def Play_Waveform(self, ucEffect):
        # Exit standby mode and use internal trigger 
        self.bus.write_byte_data(self.address, self.MODE_Reg, 0x00)

        self.bus.write_byte_data(self.address, self.WAV_SEQ1_Reg, ucEffect)
        self.bus.write_byte_data(self.address, self.WAV_SEQ2_Reg, 0x00)

        self.bus.write_byte_data(self.address, self.GO_Reg, 0x01)
        
    def close(self):
        self.bus.close()

### Step Four: 

Use I2C to play waveforms on the haptic motor

**Hardware setup:** For this demo you should connect the wires to the Arduino Pins. Connect the **`red`** wire (**`VCC`**) to the **VCC** pin, the **`black`** wire (**`GND`**) to the **GND** pin, the **`white`** wire (**`SDA`**) to **IO27**, and the **`yellow`** wire (**`SCL`**) to **IO26**.


<img src="images/i2c_hw_setup.JPG" align="center"/>

**Note:** We need to initilize the haptic motor with the i2cbus number and the bus address for the haptic motor. The i2cbus number is assigned by the linux driver when the device tree overlay is parsed and the driver run. Since there are no other i2c devices the number will be 0 i2cdetect -l can be run on the command line to see all the currenlty instantiated i2c busses and their ID numbers. The default address for the haptic motor is 0x5A.

In [6]:
import time

haptic = Haptic(0, 0x5A)

for i in range(50, 70):
    print("Effect No: " + str(i))

    haptic.Play_Waveform(i)
    time.sleep(2)

Effect No: 50
Effect No: 51
Effect No: 52
Effect No: 53
Effect No: 54
Effect No: 55
Effect No: 56
Effect No: 57
Effect No: 58
Effect No: 59
Effect No: 60
Effect No: 61
Effect No: 62
Effect No: 63
Effect No: 64
Effect No: 65
Effect No: 66
Effect No: 67
Effect No: 68
Effect No: 69
